# Data Processing

In [60]:
!pip3 install -r requirements.txt


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


In [61]:
import pandas as pd
import random
from datetime import datetime, timedelta
from faker import Faker

# Random Seed Initialization
# Set random seed to 42 for reproducibility across runs. Initialize Faker
# for generating realistic Italian names and addresses.
random.seed(42)
fake = Faker('it_IT')

## Data Extraction
### Memory-Optimized Data Loading
Load only necessary columns to save memory. MIMIC-III contains millions of records, so we optimize by loading only required fields from chartevents to reduce RAM usage and speed up processing. Other tables are small enough to load completely.

In [62]:
print("Loading MIMIC-III data...")


patients = pd.read_csv('../data/raw/PATIENTS.csv')
admissions = pd.read_csv('../data/raw/ADMISSIONS.csv')
chartevents = pd.read_csv('../data/raw/CHARTEVENTS.csv', 
                         usecols=['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM'])

print(f"Loaded {len(patients)} total patients")
print(f"Loaded {len(chartevents)} total vital events")

Loading MIMIC-III data...
Loaded 46520 total patients
Loaded 39195630 total vital events


### Random Patient Sample Selection
Select a random sample of 200 patients from the complete dataset. This creates a manageable subset for demonstration/testing purposes while maintaining statistical diversity. Using random_state ensures the same sample is selected on each run.

In [63]:
print("\nSelecting sample of 200 patients...")
sample_size = 200
sample_patients = patients.sample(n=sample_size, random_state=42)
selected_patient_ids = sample_patients['SUBJECT_ID'].tolist()

print(f"Selected {len(selected_patient_ids)} patients")


Selecting sample of 200 patients...
Selected 200 patients


### Vital Signs Item Mapping
Define the vital signs we want to track. MIMIC-III uses ITEMID codes to identify different measurements. We map these numeric codes to readable parameter names for easier processing later. These six parameters are the most common vital signs.

In [64]:
print("\nFiltering vital signs...")
vital_signs_items = {
    220045: 'heart_rate',
    220179: 'systolic_bp',
    220180: 'diastolic_bp',
    223761: 'temperature',
    220210: 'respiratory_rate',
    220277: 'spo2'
}


Filtering vital signs...


## Data Filtering

### Chart Events Filtering
Filter chartevents to keep only records for our selected patients and vital signs. This dramatically reduces the dataset size from millions to thousands of records, making subsequent processing much faster and more memory-efficient.

In [65]:
chartevents_filtered = chartevents[
    (chartevents['SUBJECT_ID'].isin(selected_patient_ids)) &
    (chartevents['ITEMID'].isin(vital_signs_items.keys()))
]

print(f"Filtered {len(chartevents_filtered)} vital events")

Filtered 36639 vital events


### Admissions Table Filtering
Filter admissions table to include only records for our selected patients. This ensures data consistency across all tables and removes unnecessary admission records that don't relate to our patient sample.

In [66]:
admissions_filtered = admissions[admissions['SUBJECT_ID'].isin(selected_patient_ids)]

## Intermediate Data Checkpoint
Save the extracted sample data to intermediate files. These files serve as checkpoints in the processing pipeline, allowing us to restart from here if needed without reprocessing the large original MIMIC-III files.

In [67]:
print("\nSaving processed data...")
sample_patients.to_csv('../data/processed/patients_sample.csv', index=False)
admissions_filtered.to_csv('../data/processed/admissions_sample.csv', index=False)
chartevents_filtered.to_csv('../data/processed/chartevents_sample.csv', index=False)

print(f"""
Done!

Statistics:
   - Patients: {len(sample_patients)}
   - Admissions: {len(admissions_filtered)}
   - Vital events: {len(chartevents_filtered)}
   
Files saved in data/processed/
""")


Saving processed data...

Done!

Statistics:
   - Patients: 200
   - Admissions: 241
   - Vital events: 36639

Files saved in data/processed/



## Data Modernization

### Extracted Sample Data Loading
Load the extracted sample data from the previous step. These files contain our filtered subset of 200 patients and their associated records, ready for temporal transformation to current dates.

In [68]:
print("Modernizing dates...")

patients = pd.read_csv('../data/processed/patients_sample.csv')
admissions = pd.read_csv('../data/processed/admissions_sample.csv')
chartevents = pd.read_csv('../data/processed/chartevents_sample.csv')

Modernizing dates...


### Time Offset Calculation
Calculate the time offset needed to shift MIMIC-III dates to present day. MIMIC-III data is historical (2001-2012), so we find the median admission date and calculate how many days to shift all dates forward to make the data current.

In [69]:
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
reference_date = admissions['ADMITTIME'].median()

current_date = datetime.now()
time_offset = current_date - reference_date

print(f"MIMIC reference date: {reference_date}")
print(f"Current date: {current_date}")
print(f"Offset to apply: {time_offset.days} days")

MIMIC reference date: 2152-07-02 14:40:00
Current date: 2025-11-02 14:52:51.413487
Offset to apply: -46263 days


### Temporal Transformation Application
Apply the calculated offset to all datetime columns across all tables. This shifts the entire dataset timeline forward, making it appear as if these hospital events occurred recently. We handle missing dates with errors='coerce' to avoid crashes.

In [70]:
print("\nApplying offset...")

admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME']) + time_offset
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME']) + time_offset
admissions['EDREGTIME'] = pd.to_datetime(admissions['EDREGTIME'], errors='coerce') + time_offset
admissions['EDOUTTIME'] = pd.to_datetime(admissions['EDOUTTIME'], errors='coerce') + time_offset

patients['DOB'] = pd.to_datetime(patients['DOB']) + time_offset
patients['DOD'] = pd.to_datetime(patients['DOD'], errors='coerce') + time_offset

chartevents['CHARTTIME'] = pd.to_datetime(chartevents['CHARTTIME']) + time_offset


Applying offset...


### Active Patient Identification
Identify currently active patients for a realistic hospital dashboard scenario. We consider patients "active" if their discharge date is in the future or missing, plus those discharged within the last 7 days for historical comparison.

In [71]:
print("\nIdentifying active patients...")

active_admissions = admissions[
    (admissions['DISCHTIME'].isna()) | 
    (admissions['DISCHTIME'] > current_date)
]

recent_discharged = admissions[
    (admissions['DISCHTIME'] <= current_date) &
    (admissions['DISCHTIME'] >= current_date - timedelta(days=7))
]


Identifying active patients...


### Working Set Assembly
Combine active and recently discharged patients to create a realistic working set. This gives us both current patients to monitor and recent history for trend analysis, mimicking what a real hospital information system would display.

In [72]:
relevant_admissions = pd.concat([active_admissions, recent_discharged])
active_patient_ids = relevant_admissions['SUBJECT_ID'].unique()

print(f"Currently admitted patients: {len(active_admissions)}")
print(f"Recently discharged patients: {len(recent_discharged)}")
print(f"Total relevant patients: {len(active_patient_ids)}")

Currently admitted patients: 121
Recently discharged patients: 0
Total relevant patients: 102


## Final Dataset Filtering

Filter all tables to include only relevant patients. This final filtering ensures we work with a cohesive dataset of currently hospitalized patients and their vital signs, eliminating historical records no longer needed for the dashboard.

In [73]:
patients_final = patients[patients['SUBJECT_ID'].isin(active_patient_ids)]
admissions_final = relevant_admissions
chartevents_final = chartevents[chartevents['SUBJECT_ID'].isin(active_patient_ids)]

## Intermediate Modernized Data Checkpoint
Save the modernized data and create a separate file listing active patient IDs. The active_patients.csv file serves as a quick reference for which patients should appear in real-time monitoring interfaces.

In [74]:
print("\nSaving modernized data...")
patients_final.to_csv('../data/processed/patients.csv', index=False)
admissions_final.to_csv('../data/processed/admissions.csv', index=False)
chartevents_final.to_csv('../data/processed/chartevents.csv', index=False)

active_patients_df = pd.DataFrame({'SUBJECT_ID': active_patient_ids})
active_patients_df.to_csv('../data/processed/active_patients.csv', index=False)

print(f"""
Done!

Statistics:
   - Active patients: {len(patients_final)}
   - Relevant admissions: {len(admissions_final)}
   - Vital events: {len(chartevents_final)}
   
Files saved in data/processed/
""")


Saving modernized data...

Done!

Statistics:
   - Active patients: 102
   - Relevant admissions: 121
   - Vital events: 16396

Files saved in data/processed/



## Anonymization and Enrichment

Load the processed data from the previous step. At this point we have current-dated records for active patients, but they still use MIMIC-III's anonymized IDs. Now we'll replace these with realistic fake identities and clinical details.

In [75]:
print("Anonymizing and enriching data...")

patients = pd.read_csv('../data/processed/patients.csv')
admissions = pd.read_csv('../data/processed/admissions.csv')

Anonymizing and enriching data...


### Patient Identity Generation
Generate unique patient identifiers and fake Italian names. This replaces MIMIC's SUBJECT_ID with more readable PT000001-style IDs and assigns realistic names using Faker library, making the dashboard look like a real Italian hospital system.

In [76]:
print("\nGenerating patient identities...")

patients['patient_id'] = [f"PT{str(i).zfill(6)}" for i in range(1, len(patients)+1)]
patients['full_name'] = [fake.name() for _ in range(len(patients))]

patients['first_name'] = patients['full_name'].apply(lambda x: x.split()[0])
patients['last_name'] = patients['full_name'].apply(lambda x: ' '.join(x.split()[1:]))


Generating patient identities...


### Hospital Location Assignment
Add hospital location details for each patient. We generate realistic room numbers using a floor-wing-room format (e.g., 3A-205), and assign patients to various medical departments to simulate a multi-specialty hospital environment.

In [77]:
print("Adding hospital details...")

departments = ['Cardiologia', 'Medicina Interna', 'Terapia Intensiva', 
               'Chirurgia', 'Pneumologia', 'Neurologia']

def generate_room():
    floor = random.randint(2, 5)
    wing = random.choice(['A', 'B', 'C'])
    room_num = random.randint(201, 250)
    return f"{floor}{wing}-{room_num}"

patients['room'] = [generate_room() for _ in range(len(patients))]
patients['department'] = [random.choice(departments) for _ in range(len(patients))]

Adding hospital details...


Create a pool of medical staff and assign each patient to a doctor and nurse. We create 20 doctors and 40 nurses (realistic ratio), then randomly assign them to patients. This adds realism and enables potential staff-based filtering.

In [78]:
print("Generating medical staff...")

doctors = [f"Dr. {fake.name()}" for _ in range(20)]
nurses = [fake.name() for _ in range(40)]

patients['attending_physician'] = [random.choice(doctors) for _ in range(len(patients))]
patients['primary_nurse'] = [random.choice(nurses) for _ in range(len(patients))]

Generating medical staff...


Assign common diagnoses to each patient. These are realistic conditions frequently seen in hospital settings, with some patients having multiple comorbidities. This enables disease-based analysis and filtering in the dashboard.

In [79]:
print("Adding diagnoses...")

common_diagnoses = [
    ['Ipertensione'],
    ['Diabete tipo 2'],
    ['Insufficienza cardiaca'],
    ['BPCO'],
    ['Polmonite'],
    ['Infarto miocardico'],
    ['Ictus ischemico'],
    ['Insufficienza renale'],
    ['Ipertensione', 'Diabete tipo 2'],
    ['Insufficienza cardiaca', 'Ipertensione'],
    ['BPCO', 'Polmonite'],
]

patients['diagnoses'] = [random.choice(common_diagnoses) for _ in range(len(patients))]

Adding diagnoses...


Assign medication regimens appropriate for the diagnoses. These medication combinations are medically realistic (e.g., diabetics get metformin, cardiac patients get heart medications), adding clinical authenticity to the dataset.

In [80]:
print("Adding medication therapies...")

common_medications = [
    ['Aspirina', 'Ramipril'],
    ['Metformina', 'Atorvastatina'],
    ['Furosemide', 'Bisoprololo'],
    ['Warfarin', 'Digoxin'],
    ['Insulina', 'Metformina'],
    ['Antibiotici', 'Cortisonici'],
]

patients['medications'] = [random.choice(common_medications) for _ in range(len(patients))]

Adding medication therapies...


Add allergy information for patient safety. Most patients have no allergies, but some have common drug or material allergies that medical staff need to know about before prescribing medications or performing procedures.

In [81]:
print("Adding allergies...")

possible_allergies = ['Nessuna', 'Penicillina', 'Lattice', 'Aspirina', 'Morfina']
patients['allergies'] = [random.choice(possible_allergies) for _ in range(len(patients))]

Adding allergies...


Generate emergency contact information for each patient. This includes a fake name, relationship, and phone number, which is critical information hospitals must maintain for contacting family members in case of emergencies.

In [82]:
print("Adding emergency contacts...")

relations = ['Moglie', 'Marito', 'Figlio', 'Figlia', 'Madre', 'Padre', 'Sorella', 'Fratello']

def generate_emergency_contact():
    return {
        'name': fake.name(),
        'relation': random.choice(relations),
        'phone': fake.phone_number()
    }

patients['emergency_contact'] = [str(generate_emergency_contact()) for _ in range(len(patients))]

Adding emergency contacts...


### Clinical Status Calculation

Calculate baseline vital signs and clinical status for each patient. We analyze their historical vital signs from chartevents to determine normal ranges and current status (stable/warning/critical) based on heart rate and blood pressure thresholds.

In [83]:
print("Calculating patient clinical status...")

chartevents = pd.read_csv('../data/processed/chartevents.csv')

patient_status = []

for _, patient in patients.iterrows():
    subject_id = patient['SUBJECT_ID']
    
    patient_vitals = chartevents[chartevents['SUBJECT_ID'] == subject_id]
    
    # If no vital signs data exists, use default normal values and mark as stable.
    # Otherwise, calculate average values from historical data to establish baseline
    # ranges that can be used for anomaly detection in real-time monitoring.
    if len(patient_vitals) == 0:
        status = 'stable'
        hr_baseline = 75
        bp_sys_baseline = 120
        bp_dia_baseline = 80
    else:
        hr_data = patient_vitals[patient_vitals['ITEMID'] == 220045]['VALUENUM']
        bp_sys_data = patient_vitals[patient_vitals['ITEMID'] == 220179]['VALUENUM']
        bp_dia_data = patient_vitals[patient_vitals['ITEMID'] == 220180]['VALUENUM']
        
        hr_baseline = hr_data.mean() if len(hr_data) > 0 else 75
        bp_sys_baseline = bp_sys_data.mean() if len(bp_sys_data) > 0 else 120
        bp_dia_baseline = bp_dia_data.mean() if len(bp_dia_data) > 0 else 80
        
        # Classify patient status based on vital sign thresholds. Critical patients
        # have significantly elevated heart rate or blood pressure, warning patients
        # show moderate elevation, and stable patients have normal values.
        if hr_baseline > 100 or bp_sys_baseline > 140:
            status = 'critical'
        elif hr_baseline > 85 or bp_sys_baseline > 130:
            status = 'warning'
        else:
            status = 'stable'
    
    patient_status.append({
        'patient_id': patient['patient_id'],
        'status': status,
        'baseline_hr': round(hr_baseline, 1),
        'baseline_bp_sys': round(bp_sys_baseline, 1),
        'baseline_bp_dia': round(bp_dia_baseline, 1)
    })

status_df = pd.DataFrame(patient_status)

Calculating patient clinical status...


### Status Data Integration

Merge the calculated status and baselines back into the patients dataframe. This consolidates all patient information into a single comprehensive table with demographics, clinical details, and current status.

In [84]:
patients = patients.merge(status_df, on='patient_id', how='left')

## Final Dataset Export
Save the final enriched dataset. This represents the complete, ready-to-use dataset with realistic patient identities, clinical information, and vital sign baselines that can be used for a hospital monitoring dashboard.

In [ ]:
print("\nSaving final data...")
patients.to_csv('../data/final/patients_enriched.csv', index=False)
admissions.to_csv('../data/final/admissions.csv', index=False)

print(f"""
Done!

Statistics:
   - Stable patients: {len(patients[patients['status'] == 'stable'])}
   - Warning patients: {len(patients[patients['status'] == 'warning'])}
   - Critical patients: {len(patients[patients['status'] == 'critical'])}
   
Files saved in data/final/
""")


Saving final data...

Done!

Statistics:
   - Stable patients: 80
   - Warning patients: 11
   - Critical patients: 11

Files saved in data/final/

